In [ ]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df_menu = pd.read_excel('./menu.xlsx',
                        header=0,
                        dtype=str).fillna('')

list_url = df_menu['Url'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import random
import requests
import json

# = = = = = = = = = = = = = = =

import time

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        part = url.split('?')[0][-12:]
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(11):
            try:
                list_appid = ['tfy6d12dc-19a5-4093-9f00-f276d950f77',
                              'picheng11-5163-4258-be21-a5629527a07',
                              'Shanghai-b2aa-4eed-82c3-b1e42e2e9179',
                              'danjidin-f8b1-4e84-adce-026944025ceb',
                              'ibay365a-b93e-46e7-8b46-6a33b716ab2a',
                              '-IBAY365-PRD-08e35c535-85b731b3']

                appid_index = random.randint(0, 5)

                developer = 'http://106.75.138.2/index.php/api/GetAppToken?auth=757F054AA22E1711B598A89EB276EDC193933EC8&appid=' + list_appid[appid_index]
                resp = requests.get(developer).text
                token = json.loads(resp)['getapptokenResponse']['token']
                
                # = = = = = = = = = = = = = = =
                
                headers = {'X-EBAY-API-IAF-TOKEN': token}
                resp = requests.get('https://open.api.ebay.com/shopping?callname=GetSingleItem&appid=' + list_appid[appid_index] + '&responseencoding=JSON&siteid=0&version=967&IncludeSelector=Details,Compatibility,ItemSpecifics&ItemID=' + part, headers=headers, proxies=Proxy.get_Proxy_Requests()).text

                info = json.loads(resp)
                if info['Ack'] != 'Success':
                    continue
                
                # = = = = = = = = = = = = = = =
                
                title = info['Item']['Title']
                
                # = = = = = = = = = = = = = = =
                
                price = info['Item']['CurrentPrice']['CurrencyID'] + ' ' + str(info['Item']['CurrentPrice']['Value'])

                # = = = = = = = = = = = = = = =
                
                sold = info['Item']['QuantitySold']
                
                # = = = = = = = = = = = = = = =
                
                if 'ItemCompatibilityList' in info['Item']:
                    vehicle = info['Item']['ItemCompatibilityList']['Compatibility']
                else:
                    vehicle = ''
                    
                # = = = = = = = = = = = = = = =
                
                dict_vehicle = {}
                for list_name_value in vehicle:
                    make = ''
                    model = ''
                    year = ''
                    for name_value in list_name_value['NameValueList']:
                        if 'Name' in name_value and 'Make' in name_value['Name']:
                            make = name_value['Value'][0]
                        elif 'Name' in name_value and 'Model' in name_value['Name']:
                            model = name_value['Value'][0]
                        elif 'Name' in name_value and 'Year' in name_value['Name']:
                            year = int(name_value['Value'][0])

                    if make not in dict_vehicle:
                        dict_vehicle[make] = {model: [year]}
                    else:
                        if model not in dict_vehicle[make]:
                            dict_vehicle[make][model] = [year]
                        else:
                            dict_vehicle[make][model].append(year)

                list_vehicle = []
                for make in dict_vehicle:
                    for model in dict_vehicle[make]:
                        list_vehicle.append(make + ' ' + model + ' ' + str(min(dict_vehicle[make][model])) + '-' + str(max(dict_vehicle[make][model])))

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                if 'GalleryURL' in info['Item']:
                    src = info['Item']['GalleryURL']
                elif 'PictureURL' in info['Item']:
                    if len(info['Item']['PictureURL']) == 0:
                        src = ''
                    else:
                        src = info['Item']['PictureURL'][0]
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Item_Number': part,
                                         'Title': title,
                                         'Price': price,
                                         'Sold': sold,
                                         'Vehicle': vehicle,
                                         'Pic': '',
                                         'Url': url,
                                         'Src': src}])
                
                status = 'ok'
                
                # = = = = = = = = = = = = = = =
                
                if 'ItemSpecifics' in info['Item']:
                    details = info['Item']['ItemSpecifics']['NameValueList']
                else:
                    details = ''
                
                # = = = = = = = = = = = = = = =
                
                for name_value in details:
                    df_temp[name_value['Name'].lower()] = ';'.join(name_value['Value'])
                    
                break
                
            except:
                time.sleep(3)
                continue
                
        # = = = = = = = = = = = = = = =

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Item_Number': part,
                                     'Url': url}])

        # = = = = = = = = = = = = = = =

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(part +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(7):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
df.to_excel('./part.xlsx', index=False)
print()
print('爬虫结束')

总数量：5000

234803749220  <->  [ok] - 剩余数量：4993
371112698702  <->  [ok] - 剩余数量：4992
351655934056  <->  [ok] - 剩余数量：4991
371682879652  <->  [ok] - 剩余数量：4990
372321518748  <->  [ok] - 剩余数量：4989
234465835816  <->  [ok] - 剩余数量：4988
233384562297  <->  [ok] - 剩余数量：4987
372634369670  <->  [ok] - 剩余数量：4986
373604487895  <->  [ok] - 剩余数量：4985
373604488033  <->  [ok] - 剩余数量：4984
351773287081  <->  [ok] - 剩余数量：4983
372544053034  <->  [ok] - 剩余数量：4982
354065389760  <->  [ok] - 剩余数量：4981
354725753454  <->  [ok] - 剩余数量：4980
351615196189  <->  [ok] - 剩余数量：4979
371496564898  <->  [ok] - 剩余数量：4978
372562497163  <->  [ok] - 剩余数量：4977
354065389807  <->  [ok] - 剩余数量：4976
353523607863  <->  [ok] - 剩余数量：4975
374523622414  <->  [ok] - 剩余数量：4974
353577009128  <->  [ok] - 剩余数量：4973
372321482425  <->  [ok] - 剩余数量：4972
352366953958  <->  [ok] - 剩余数量：4971
352564260522  <->  [ok] - 剩余数量：4970
353576981490  <->  [ok] - 剩余数量：4969
233072108052  <->  [ok] - 剩余数量：4968
372151454308  <->  [ok] - 剩余数量：4967
233088264292  <-> 

234091504154  <->  [ok] - 剩余数量：4765
350871780104  <->  [ok] - 剩余数量：4764
371555935663  <->  [ok] - 剩余数量：4763
373651402195  <->  [ok] - 剩余数量：4762
352217206630  <->  [ok] - 剩余数量：4761
232781459096  <->  [ok] - 剩余数量：4760
373429425200  <->  [ok] - 剩余数量：4759
354597671758  <->  [ok] - 剩余数量：4758
352175577786  <->  [ok] - 剩余数量：4757
234092851016  <->  [ok] - 剩余数量：4756
353972841493  <->  [ok] - 剩余数量：4755
353959505546  <->  [ok] - 剩余数量：4754
233069664821  <->  [ok] - 剩余数量：4753
352566875099  <->  [ok] - 剩余数量：4752
233323775586  <->  [ok] - 剩余数量：4751
234041089745  <->  [ok] - 剩余数量：4750
374307060881  <->  [ok] - 剩余数量：4749
352217230369  <->  [ok] - 剩余数量：4748
233089544916  <->  [ok] - 剩余数量：4747
373308795087  <->  [ok] - 剩余数量：4746
234797630661  <->  [ok] - 剩余数量：4745
373786537032  <->  [ok] - 剩余数量：4744
374492525416  <->  [ok] - 剩余数量：4743
233125685654  <->  [ok] - 剩余数量：4742
373666777995  <->  [ok] - 剩余数量：4741
372562531831  <->  [ok] - 剩余数量：4740
233105915510  <->  [ok] - 剩余数量：4739
351773365185  <->  [ok] - 剩余